In [1]:
import sys
import math
import pickle
import numpy as np
import pandas as pd
import networkx as nx

#===读取数据集===
task = pd.read_pickle(r"outData/task-2.pkl")
func = pd.read_pickle(r"outData/func.pkl")
with open(r"outData/graph.pkl", "rb") as file:
    graph = pickle.load(file)
with open(r"outData/user.pkl", "rb") as file:
    user = pickle.load(file)
with open(r"outData/node.pkl", "rb") as file:
    node = pickle.load(file)

In [2]:
'''
##################################
#----------各种工具类-----------
##################################
'''


def auto_str(cls):

    def __str__(self):
        return '%s(%s)' % (type(self).__name__, ', '.join(
            '%s=%s' % item for item in vars(self).items()))

    cls.__str__ = __str__
    return cls


#1.任务类
@auto_str
class Task:

    def __init__(self, taskId, arvTime, execTime, softDdl, dataVol, cpuCore,
                 instNum, funcId, serviceId):
        self.taskId = taskId  #任务Id
        self.arvTime = arvTime  #到达时间
        self.execTime = execTime  #执行时间
        self.softDdl = softDdl  #软截止期
        self.dataVol = dataVol  #数据量
        self.cpuCore = cpuCore  #Cpu核心数
        self.instNum = instNum  #实例个数
        self.funcId = funcId  #函数Id
        self.serviceId = serviceId  #服务商Id


#2.边缘节点类
@auto_str
class Node:

    def __init__(self, nodeId, serviceId, cpuCore, cacheCapacity, funcSet,
                 bandwidth, recv_queue, wait_queue, exec_queue, tran_queue):
        self.nodeId = nodeId  #节点Id
        self.serviceId = serviceId  #服务商Id
        self.cpuCore = cpuCore  #Cpu核心数
        self.cacheCapacity = cacheCapacity  #缓存容量
        self.funcSet = funcSet  #函数集合
        self.bandwidth = bandwidth  #固定带宽
        self.recv_queue = recv_queue  #接收队列:接收用户传输过来的任务
        self.wait_queue = wait_queue  #等待队列
        self.exec_queue = exec_queue  #执行队列
        self.tran_queue = tran_queue  #传输队列:接收其它边缘节点传输过来的任务+冷启动完成的任务

In [3]:
'''
##################################
#----------各种工具函数-----------
##################################
'''


#1.从func中按比例随机生成本次实验的函数集
def getFuncSet(func_num):
    net = getFuncSetByType(".net", func_num)
    golang = getFuncSetByType("golang", func_num)
    python = getFuncSetByType("python", func_num)
    nodejs = getFuncSetByType("node.js", func_num)
    java = getFuncSetByType("java", func_num)
    php = getFuncSetByType("php", func_num)
    res = np.vstack((net, golang, python, nodejs, java, php))
    return np.delete(res, np.s_[2:2 + len(res) - func_num], axis=0)


def getFuncSetByType(runc_type, func_num):
    n = len(func)
    subFunc = func[func["runc_type"] == runc_type]
    m = len(subFunc)
    num = int(np.round(func_num * (m / n)))
    num = num if num >= 1 else 1
    #print(runc_type, num)
    return subFunc.sample(n=num, replace=False).to_numpy()

In [8]:
#===初始化超参数===
lambda_rate = 10
cost_diff = 1.1  #不同服务提供商之间的传输代价
func_num = 20  #函数集中的函数个数
cpu_core = 50.0
cache_capacity = 256 * 1024.0  #暂时假设相同
soft_ddl_param = [0.25, 0.50]

n = 125
bandwidth_sub_1 = 1000.0
bandwidth_sub_6 = 2000.0
bandwidth_mmWave_24 = 4000.0
func_set = getFuncSet(func_num)
task['soft_ddl'] = task['exec_time'].apply(lambda x: np.ceil(
    max(1.0,
        np.random.uniform(soft_ddl_param[0], soft_ddl_param[1]) * x)))

In [6]:
func_set

array([['.net', 1.0, 512.0],
       ['golang', 1.0, 3072.0],
       ['python', 1.0, 512.0],
       ['python', 4.0, 512.0],
       ['python', 1.0, 128.0],
       ['python', 3.0, 256.0],
       ['python', 2.0, 256.0],
       ['python', 4.0, 512.0],
       ['python', 1.0, 512.0],
       ['python', 1.0, 128.0],
       ['python', 3.0, 512.0],
       ['node.js', 18.0, 256.0],
       ['node.js', 7.0, 512.0],
       ['node.js', 1.0, 4096.0],
       ['node.js', 10.0, 512.0],
       ['node.js', 1.0, 3072.0],
       ['node.js', 2.0, 256.0],
       ['java', 3.0, 1216.0],
       ['java', 5.0, 1024.0],
       ['php', 1.0, 128.0]], dtype=object)

In [136]:
#===初始化每个基站的CPU核数|缓存容量|函数集合|网络带宽===
sub_1 = np.array(
    [30, 89, 52, 57, 93, 53, 1, 56, 12, 116, 38, 50, 86, 76, 84, 28, 22, 98])
sub_6 = np.array([
    103, 54, 108, 45, 95, 36, 58, 14, 6, 11, 0, 96, 88, 25, 104, 113, 80, 81,
    94, 7, 31, 92, 115, 39, 21, 72, 4, 9, 105, 85, 37, 63, 107, 17, 79, 26
])
mmWave_24 = np.array([
    32, 55, 124, 118, 101, 3, 106, 97, 66, 19, 68, 13, 122, 70, 23, 59, 15, 47,
    10, 87, 102, 65, 29, 2, 73, 5, 75, 114, 117, 77, 33, 35, 60, 8, 46, 111,
    112, 71, 43, 61, 27, 51, 69, 121, 90, 16, 41, 64, 67, 34, 99, 120, 123, 62,
    100, 74, 109, 42, 119, 48, 78, 24, 83, 110, 20, 44, 91, 40, 82, 18, 49
])
node_list = []
for i in np.arange(n):
    if i in sub_1:
        node_list.append(
            Node(nodeId=i,
                 serviceId=1,
                 cpuCore=cpu_core,
                 cacheCapacity=cache_capacity,
                 funcSet={},
                 bandwidth=bandwidth_sub_1,
                 recv_queue=[],
                 wait_queue=[],
                 exec_queue=[],
                 tran_queue=[]))
    elif i in sub_6:
        node_list.append(
            Node(nodeId=i,
                 serviceId=6,
                 cpuCore=cpu_core,
                 cacheCapacity=cache_capacity,
                 funcSet={},
                 bandwidth=bandwidth_sub_6,
                 recv_queue=[],
                 wait_queue=[],
                 exec_queue=[],
                 tran_queue=[]), )
    else:  # i in mmWave_24
        node_list.append(
            Node(nodeId=i,
                 serviceId=24,
                 cpuCore=cpu_core,
                 cacheCapacity=cache_capacity,
                 funcSet={},
                 bandwidth=bandwidth_mmWave_24,
                 recv_queue=[],
                 wait_queue=[],
                 exec_queue=[],
                 tran_queue=[]))
node_list = np.array(node_list)

In [137]:
#===初始化新的Sub_1|Sub_6|mmWave_24|Cross_net基站拓扑图===
G_sub_1 = graph["sub-1"]
G_sub_6 = graph["sub-6"]
G_mmWave_24 = graph["mmWave-24"]
G_cross_net = graph["cross-net"]

for u, v, _ in G_sub_1.edges(data=True):
    G_sub_1[u][v]["weight"] = 1.0 / bandwidth_sub_1

for u, v, _ in G_sub_6.edges(data=True):
    G_sub_6[u][v]["weight"] = 1.0 / bandwidth_sub_6

for u, v, _ in G_mmWave_24.edges(data=True):
    G_mmWave_24[u][v]["weight"] = 1.0 / bandwidth_mmWave_24

for u, v, _ in G_cross_net.edges(data=True):
    if u in sub_1:
        bandwidth = bandwidth_sub_1
    elif u in sub_6:
        bandwidth = bandwidth_sub_6
    else:
        bandwidth = bandwidth_mmWave_24
    if (u in sub_1 and v in sub_1) or (u in sub_6
                                       and v in sub_6) or (u in mmWave_24
                                                           and v in mmWave_24):
        G_cross_net[u][v]["weight"] = 1.0 / bandwidth
    else:
        G_cross_net[u][v]["weight"] = cost_diff / bandwidth

G = {}
G[1] = G_sub_1
G[6] = G_sub_6
G[24] = G_mmWave_24
G["cross-net"] = G_cross_net

In [1]:
'''
##################################
#----------节点选择策略-----------#
##################################
'''


#min_dis_own:选择距离最近的本服务提供商的基站
def NS_min_dis_own(userId, funcId=-1):
    serviceId = 1 if userId % 3 == 1 else (6 if userId % 3 == 2 else 24)
    sortedNdLst = sorted(user[userId], key=lambda x: x[2])
    for nd in sortedNdLst:
        if nd[1] == serviceId:
            return nd[0]


#min_dis_all:选择距离最近的基站
def NS_min_dis_all(userId, funcId=-1):
    return min(user[userId], key=lambda x: x[2])[0]


#min_user:选择覆盖用户数最少的基站
def NS_min_user(userId, funcId=-1):
    return min(user[userId], key=lambda x: node[x[0]][0])[0]


#max_node:选择邻居基站数最多的基站
def NS_max_node(userId, funcId=-1):
    return max(user[userId], key=lambda x: node[x[0]][1])[0]


#max_cpu:选择负载最低（剩余CPU核心数最多）的基站
def NS_max_cpu(userId, funcId=-1):
    return max(user[userId], key=lambda x: node_list[x[0]].cpuCore)[0]


#max_cache:选择剩余缓存容量最多的基站
def NS_max_cache(userId, funcId=-1):
    return max(user[userId], key=lambda x: node_list[x[0]].cacheCapacity)[0]


#exist_func:选择距离最近的存在函数的基站
def NS_exist_func(userId, funcId):
    sortedNdLst = sorted(user[userId], key=lambda x: x[2])
    for nd in sortedNdLst:
        if funcId in node_list[nd[0]].funcSet:
            return nd[0]
    return sortedNdLst[0][0]


NS = {
    "NS_min_dis_own": NS_min_dis_own,
    "NS_min_dis_all": NS_min_dis_all,
    "NS_min_user": NS_min_user,
    "NS_max_node": NS_max_node,
    "NS_max_cpu": NS_max_cpu,
    "NS_max_cache": NS_max_cache,
    "NS_exist_func": NS_exist_func
}

# print(NS_min_dis_own(0))
# print(NS_min_dis_all(0))
# print(NS_min_user(0))
# print(NS_max_node(0))
# print(NS_max_cpu(0))
# print(NS_exist_func(0, 1))
# print(NS_exist_func(0, 0))

In [2]:
'''
##################################
#----------路径选择策略-----------#
##################################
'''


#min_dis:寻找可以处理任务task的距离节点nodeId最近的边缘节点
def PS_min_dis(G, nodeId, task):
    #按照距离边缘节点nodeId的最短路径大小正序排序
    for tarNd, count in nx.single_source_dijkstra_path_length(G,
                                                              nodeId).items():
        if tarNd == nodeId:
            continue
        #print(tarNd, count)
        tarNd = node_list[tarNd]
        if (task.cpuCore <= tarNd.cpuCore) and (
                task.instNum * func_set[task.funcId][2]
                <= tarNd.cacheCapacity) and (task.funcId in tarNd.funcSet):
            return tarNd.nodeId, task.dataVol * count
    return -1, np.finfo(np.float32).max


#max_cpu:寻找可以处理任务task的剩余Cpu核心数最多的边缘节点
def PS_max_cpu(G, nodeId, task):
    #按照剩余Cpu核心数倒序排序,如果相同按照距离边缘节点nodeId的最短路径大小正序排序
    sortedNdlst = sorted(
        G.nodes(),
        key=lambda x: (-node_list[x].cpuCore,
                       nx.shortest_path_length(G, nodeId, x, weight="weight")))
    for tarNd in sortedNdlst:
        if tarNd == nodeId:
            continue
        tarNd = node_list[tarNd]
        if (task.cpuCore <= tarNd.cpuCore) and (
                task.instNum * func_set[task.funcId][2]
                <= tarNd.cacheCapacity) and (task.funcId in tarNd.funcSet):
            return tarNd.nodeId, task.dataVol * nx.shortest_path_length(
                G, nodeId, tarNd.nodeId, weight="weight")
    return -1, np.finfo(np.float32).max


#max_cache:寻找可以处理任务task的剩余缓存容量最多的边缘节点
def PS_max_cache(G, nodeId, task):
    #按照剩余缓存容量倒序排序,如果相同按照距离边缘节点nodeId的最短路径大小正序排序
    sortedNdlst = sorted(
        G.nodes(),
        key=lambda x: (-node_list[x].cacheCapacity,
                       nx.shortest_path_length(G, nodeId, x, weight="weight")))
    for tarNd in sortedNdlst:
        if tarNd == nodeId:
            continue
        tarNd = node_list[tarNd]
        if (task.cpuCore <= tarNd.cpuCore) and (
                task.instNum * func_set[task.funcId][2]
                <= tarNd.cacheCapacity) and (task.funcId in tarNd.funcSet):
            return tarNd.nodeId, task.dataVol * nx.shortest_path_length(
                G, nodeId, tarNd.nodeId, weight="weight")
    return -1, np.finfo(np.float32).max


PS = {
    "PS_min_dis": PS_min_dis,
    "PS_max_cache": PS_max_cache,
    "PS_max_cpu": PS_max_cpu
}

In [3]:
'''
##################################
#----------任务排序策略-----------#
##################################
'''


#exec_time_asc:按照任务的执行时间升序排序
def TS_exec_time_asc(wait_queue, cur_time):
    return sorted(wait_queue, key=lambda x: x.execTime)


#exec_time_desc:按照任务的执行时间降序排序
def TS_exec_time_desc(wait_queue, cur_time):
    return sorted(wait_queue, key=lambda x: -x.execTime)


#data_vol_asc:按照任务的数据量升序排序
def TS_data_vol_asc(wait_queue, cur_time):
    return sorted(wait_queue, key=lambda x: x.dataVol)


#data_vol_desc:按照任务的数据量降序排序
def TS_data_vol_desc(wait_queue, cur_time):
    return sorted(wait_queue, key=lambda x: -x.dataVol)


#exec_time_to_data_vol_ratio_asc:按照任务的执行时间/数据量升序排序
def TS_exec_time_to_data_vol_ratio_asc(wait_queue, cur_time):
    return sorted(wait_queue, key=lambda x: x.execTime / x.dataVol)


#exec_time_to_data_vol_ratio_desc:按照任务的执行时间/数据量降序排序
def TS_exec_time_to_data_vol_ratio_desc(wait_queue, cur_time):
    return sorted(wait_queue, key=lambda x: -x.execTime / x.dataVol)


#closest_soft_ddl:按照任务的软截止期逼近程度排序
def TS_closest_soft_ddl(wait_queue, cur_time):
    return sorted(wait_queue, key=lambda x: x.arvTime + x.softDdl - cur_time)


#highest_response_ratio:按照任务的响应比降序排序
def TS_highest_response_ratio(wait_queue, cur_time):
    return sorted(
        wait_queue,
        key=lambda x: -(x.execTime + cur_time - x.arvTime) / x.execTime)


TS = {
    "TS_exec_time_asc": TS_exec_time_asc,
    "TS_exec_time_desc": TS_exec_time_desc,
    "TS_data_vol_asc": TS_data_vol_asc,
    "TS_data_vol_desc": TS_data_vol_desc,
    "TS_exec_time_to_data_vol_ratio_asc": TS_exec_time_to_data_vol_ratio_asc,
    "TS_exec_time_to_data_vol_ratio_desc": TS_exec_time_to_data_vol_ratio_desc,
    "TS_closest_soft_ddl": TS_closest_soft_ddl,
    "TS_highest_response_ratio": TS_highest_response_ratio
}

In [141]:
task_num = len(task)  #任务总数
task_over_num = 0  #已完成任务
task_use_num = 0  #当前使用到数据集中的第几个任务|下次拿任务从第task_use_num个获取

t = 0

#采用不同处理方式的任务个数
Local = 0
Inter_Edge = 0
Cross_Edge = 0
Cold_Start = 0

#指定各算法组件的策略
node_selection_strategy = "NS_min_dis_all"
path_selection_strategy = "PS_min_dis"
task_sorting_strategy = "TS_closest_soft_ddl"

arrivalDic = {}
delayDic = {}

#在每一时隙开始时进行调度
while task_over_num < task_num:

    #print("now time slot", t)
    #边缘节点不同队列的数据格式
    #recv_queue|tram_queue->[[task,t],...]
    #wait_queue|exec_queue->[task,...]

    #在新时隙开始时对每个边缘节点的接收队列+传输队列+执行队列中的时间都减1
    for nd in np.arange(n):
        curNd = node_list[nd]
        RecvQe = curNd.recv_queue
        TranQe = curNd.tran_queue
        ExecQe = curNd.exec_queue
        for i in range(len(RecvQe) - 1, -1, -1):
            RecvQe[i][1] -= 1
        for i in range(len(TranQe) - 1, -1, -1):
            TranQe[i][1] -= 1
        for i in range(len(ExecQe) - 1, -1, -1):
            ExecQe[i].execTime -= 1
            #不仅执行时间-1,还要恢复Cpu核心数+缓存容量+删除任务+删除函数
            if ExecQe[i].execTime == 0:
                curNd.funcSet[ExecQe[i].funcId] -= 1
                if curNd.funcSet[ExecQe[i].funcId] == 0:
                    del curNd.funcSet[ExecQe[i].funcId]
                curNd.cpuCore += ExecQe[i].cpuCore
                curNd.cacheCapacity += ExecQe[i].instNum * func_set[
                    ExecQe[i].funcId][2]
                task_over_num += 1
                delayDic[ExecQe[i].taskId] = max(
                    t - arrivalDic[ExecQe[i].taskId], 0)
                del ExecQe[i]

    #依旧存在没有到来的任务
    if task_use_num < task_num:
        #每个用户开始生成任务
        for u in user:
            if task_use_num >= task_num:
                break
            #该用户在t时隙生成的任务个数
            utNum = np.random.poisson(lambda_rate)
            #print("user:", u, "->", utNum, "tasks")
            #print(min(user[u], key=lambda x: x[2]))
            for i in np.arange(utNum):
                if task_use_num >= task_num:
                    break
                #构建任务对象
                tk = Task(taskId=task_use_num,
                          arvTime=t,
                          execTime=task.iloc[task_use_num, 0],
                          softDdl=task.iloc[task_use_num, 4],
                          dataVol=task.iloc[task_use_num, 1],
                          cpuCore=task.iloc[task_use_num, 3],
                          instNum=task.iloc[task_use_num, 2],
                          funcId=np.random.randint(0, func_num),
                          serviceId=1 if u % 3 == 1 else
                          (6 if u % 3 == 2 else 24))
                #记录每个任务的到达时间
                arrivalDic[tk.taskId] = t + tk.execTime + tk.softDdl
                #print("----------")
                #print(str(tk))
                '''
                #-----------------#
                #   节点选择组件   #
                #-----------------#
                '''
                selectedNode = NS[node_selection_strategy](u, tk.funcId)
                #-----------------#
                #    Recv_Queue   #
                #-----------------#
                node_list[selectedNode].recv_queue.append([
                    tk,
                    math.ceil(tk.dataVol /
                              (node_list[selectedNode].bandwidth /
                               node[selectedNode][0]))  #数据量/(基站带宽/基站覆盖用户数)
                ])
                #print("select node", selectedNode)
                #print("this node bandwidth", node_list[selectedNode].bandwidth)
                #print("this node has", node[selectedNode][0], "users")
                #print(node_list[selectedNode].recv_queue)
                #print("----------")
                task_use_num += 1
    else:
        print(task_use_num)
        break

    #对每个边缘节点接收到的到期任务进行卸载决策并放入到对应边缘节点的等待队列或传输队列
    for nd in np.arange(n):
        curNd = node_list[nd]
        RecvQe = curNd.recv_queue
        for i in range(len(RecvQe) - 1, -1, -1):
            #任务tk到达边缘节点nd
            if RecvQe[i][1] == 0:
                tk = RecvQe[i][0]
                del RecvQe[i]
                '''
                #-----------------#
                #   卸载决策组件   #
                #-----------------#
                '''
                #Local:当前边缘节点即可处理该任务(Cpu+缓存+存在函数)
                if (tk.cpuCore <= curNd.cpuCore) and (
                        tk.instNum * func_set[tk.funcId][2]
                        <= curNd.cacheCapacity) and (tk.funcId
                                                     in curNd.funcSet):
                    #-----------------#
                    #    Wait_Queue   #
                    #-----------------#
                    curNd.wait_queue.append(tk)
                    #Local方式延迟时间的计算
                    #t=结束时隙+基站将任务传回给用户时间-开始时隙-任务执行时间-任务软截止期时间
                    #t=结束时隙-(开始时隙+任务执行时间+任务软截止期时间-基站将任务传回给用户时间)
                    arrivalDic[tk.taskId] -= math.ceil(
                        (tk.dataVol / 10) /
                        (curNd.bandwidth / node[nd][0]))  #数据量/10
                    Local += 1
                    continue
                '''
                #-----------------#
                #   路径选择组件   #
                #-----------------#
                '''
                #Inter-Edge:在该基站的内网寻找其它基站处理该任务
                ieNd, ieCost = PS[path_selection_strategy](G[curNd.serviceId],
                                                           nd, tk)
                #Cross-Edge:在全网寻找其它基站处理该任务
                ceNd, ceCost = PS[path_selection_strategy](G["cross-net"], nd,
                                                           tk)
                #Cold-start:在当前边缘节点冷启动处理该任务
                csNd, csCost = nd, func_set[tk.funcId][1]
                #选择代价最小的处理方式
                offload_decision = min(
                    [[0, ieNd, ieCost], [1, ceNd, ceCost], [2, csNd, csCost]],
                    key=lambda x: x[2])
                #Inter-Edge方式延迟时间的计算
                #t=结束时隙+基站将任务传回到达基站的时间+到达基站将任务传回给用户时间-开始时隙-任务执行时间-任务软截止期时间
                #t=结束时隙-(开始时隙+任务执行时间+任务软截止期时间-基站将任务传回到达基站的时间-到达基站将任务传回给用户时间)
                if offload_decision[0] == 0:
                    arrivalDic[tk.taskId] -= math.ceil((
                        (tk.dataVol / 10) * nx.shortest_path_length(
                            G[curNd.serviceId], ieNd, nd, weight="weight")) + (
                                (tk.dataVol / 10) /
                                (curNd.bandwidth / node[nd][0])))
                    Inter_Edge += 1
                #Cross-Edge方式延迟时间的计算
                elif offload_decision[0] == 1:
                    arrivalDic[tk.taskId] -= math.ceil((
                        (tk.dataVol / 10) * nx.shortest_path_length(
                            G["cross-net"], ceNd, nd, weight="weight")) + (
                                (tk.dataVol / 10) /
                                (curNd.bandwidth / node[nd][0])))
                    Cross_Edge += 1
                #Cold-start方式延迟时间的计算
                else:  #同Local方式
                    arrivalDic[tk.taskId] -= math.ceil(
                        (tk.dataVol / 10) / (curNd.bandwidth / node[nd][0]))
                    Cold_Start += 1
                #-----------------#
                #    Tran_Queue   #
                #-----------------#
                node_list[offload_decision[1]].tran_queue.append(
                    [tk, math.ceil(offload_decision[2])])

    #将每个边缘节点卸载而来的任务+冷启动完成的任务放入执行队列
    for nd in np.arange(n):
        curNd = node_list[nd]
        TranQe = curNd.tran_queue
        for i in range(len(TranQe) - 1, -1, -1):
            if TranQe[i][1] == 0:
                tk = TranQe[i][0]
                del TranQe[i]
                #-----------------#
                #    Wait_Queue   #
                #-----------------#
                curNd.wait_queue.append(tk)

    #对每个边缘节点等待队列中的任务进行任务排序并依次放入执行队列
    for nd in np.arange(n):
        curNd = node_list[nd]
        '''
        #-----------------#
        #   任务排序组件   #
        #-----------------#
        '''
        curNd.wait_queue = TS[task_sorting_strategy](curNd.wait_queue, t)
        WaitQe = curNd.wait_queue
        for i in range(len(WaitQe) - 1, -1, -1):
            tk = WaitQe[i]
            if tk.cpuCore <= curNd.cpuCore and (tk.instNum *
                                                func_set[tk.funcId][2]
                                                <= curNd.cacheCapacity):
                #-----------------#
                #    Exec_Queue   #
                #-----------------#
                curNd.exec_queue.append(tk)
                curNd.funcSet[tk.funcId] = curNd.funcSet.get(tk.funcId, 0) + 1
                curNd.cpuCore -= tk.cpuCore
                curNd.cacheCapacity -= tk.instNum * func_set[tk.funcId][2]
                del WaitQe[i]

    print("=============================")
    print("当前时隙:", t)
    print("已到达任务数:", task_use_num)
    print(Local, Inter_Edge, Cross_Edge, Cold_Start)
    print("已完成任务数:", task_over_num)
    print("=============================")
    t += 1

print(t)
print("已到达任务数", task_use_num)
print("已完成任务数", task_over_num)
print(len(delayDic))
np.mean(list(delayDic.values()))

当前时隙: 0
已到达任务数: 8318
0 0 0 0
已完成任务数: 0
当前时隙: 1
已到达任务数: 16455
0 0 0 2277
已完成任务数: 0
当前时隙: 2
已到达任务数: 24668
0 0 0 6668
已完成任务数: 0
当前时隙: 3
已到达任务数: 32970
2136 711 132 8581
已完成任务数: 0
当前时隙: 4
已到达任务数: 41147
5077 1163 345 10190
已完成任务数: 40
当前时隙: 5
已到达任务数: 49439
8353 1702 910 11304
已完成任务数: 410
当前时隙: 6
已到达任务数: 57659
12198 2315 1765 11875
已完成任务数: 1504
当前时隙: 7
已到达任务数: 65816
16101 2923 2586 12483
已完成任务数: 2552
当前时隙: 8
已到达任务数: 73952
20287 3732 3477 12800
已完成任务数: 3547
当前时隙: 9
已到达任务数: 81994
24492 4765 4761 12800
已完成任务数: 4515
当前时隙: 10
已到达任务数: 90223
28866 5696 6242 12800
已完成任务数: 5483
当前时隙: 11
已到达任务数: 98598
33375 6661 7552 12800
已完成任务数: 6455
当前时隙: 12
已到达任务数: 106523
37870 7718 8832 12800
已完成任务数: 7466
当前时隙: 13
已到达任务数: 114859
42459 8711 10119 12800
已完成任务数: 8559
当前时隙: 14
已到达任务数: 123004
46988 9745 11508 12800
已完成任务数: 9580
当前时隙: 15
已到达任务数: 131270
51545 10760 12916 12800
已完成任务数: 10502
当前时隙: 16
已到达任务数: 139385
56236 11807 14332 12800
已完成任务数: 11510
当前时隙: 17
已到达任务数: 147587
60832 12794 15832 12800
已完成任务数: 12463
当前时隙: 18


1.8583222112197013

In [2]:
#2.417519212645856
#1.8583222112197013

In [2]:
res=pd.read_csv(r"out.csv", sep="|")
res.head(10)

,2,1.2,10,100,256,"[0.1, 0.2]",NS_min_dis_own,PS_min_dis,TS_exec_time_asc,1
0,2,1.2,10,100,256,"[0.1, 0.2]",NS_min_dis_own,PS_min_dis,TS_data_vol_asc,1
1,2,1.2,10,100,256,"[0.1, 0.2]",NS_min_dis_own,PS_min_dis,TS_exec_time_to_data_vol_ratio_asc,1
2,2,1.2,10,100,256,"[0.1, 0.2]",NS_min_dis_own,PS_min_dis,TS_closest_soft_ddl,1
3,2,1.2,10,100,256,"[0.1, 0.2]",NS_min_dis_own,PS_min_dis,TS_highest_response_ratio,1
4,2,1.2,10,100,256,"[0.1, 0.2]",NS_min_dis_own,PS_max_cache,TS_exec_time_asc,1
5,2,1.2,10,100,256,"[0.1, 0.2]",NS_min_dis_own,PS_max_cache,TS_data_vol_asc,1
6,2,1.2,10,100,256,"[0.1, 0.2]",NS_min_dis_own,PS_max_cache,TS_exec_time_to_data_vol_ratio_asc,1
7,2,1.2,10,100,256,"[0.1, 0.2]",NS_min_dis_own,PS_max_cache,TS_closest_soft_ddl,1
8,2,1.2,10,100,256,"[0.1, 0.2]",NS_min_dis_own,PS_max_cache,TS_highest_response_ratio,1
9,2,1.2,10,100,256,"[0.1, 0.2]",NS_min_dis_own,PS_max_cpu,TS_exec_time_asc,1


In [3]:
res.tail(10)

,2,1.2,10,100,256,"[0.1, 0.2]",NS_min_dis_own,PS_min_dis,TS_exec_time_asc,1
749989,10,2.0,50,500,1024,"[0.4, 0.5]",NS_exist_func,PS_max_cache,TS_exec_time_asc,1
749990,10,2.0,50,500,1024,"[0.4, 0.5]",NS_exist_func,PS_max_cache,TS_data_vol_asc,1
749991,10,2.0,50,500,1024,"[0.4, 0.5]",NS_exist_func,PS_max_cache,TS_exec_time_to_data_vol_ratio_asc,1
749992,10,2.0,50,500,1024,"[0.4, 0.5]",NS_exist_func,PS_max_cache,TS_closest_soft_ddl,1
749993,10,2.0,50,500,1024,"[0.4, 0.5]",NS_exist_func,PS_max_cache,TS_highest_response_ratio,1
749994,10,2.0,50,500,1024,"[0.4, 0.5]",NS_exist_func,PS_max_cpu,TS_exec_time_asc,1
749995,10,2.0,50,500,1024,"[0.4, 0.5]",NS_exist_func,PS_max_cpu,TS_data_vol_asc,1
749996,10,2.0,50,500,1024,"[0.4, 0.5]",NS_exist_func,PS_max_cpu,TS_exec_time_to_data_vol_ratio_asc,1
749997,10,2.0,50,500,1024,"[0.4, 0.5]",NS_exist_func,PS_max_cpu,TS_closest_soft_ddl,1
749998,10,2.0,50,500,1024,"[0.4, 0.5]",NS_exist_func,PS_max_cpu,TS_highest_response_ratio,1
